In [1]:
import import_ipynb
#import RNN
#import XGBOOST
#import MLR
#import MLoR
import Decision_Tree
import Visualization
import pickle
import pandas as pd
import numpy as np
import keras
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import math

from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

from keras import Sequential, optimizers
from keras.layers import Dense, LSTM, Dropout

from sklearn.metrics import mean_absolute_error

from keras import backend as K

from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from math import sqrt


importing Jupyter notebook from Decision_Tree.ipynb


Using TensorFlow backend.


importing Jupyter notebook from Visualization.ipynb


In [3]:
class Hurricane:
    def __init__(self, ocean, id_no, year, name, length, hurr_data ):
        self.ocean = ocean
        self.id_no = id_no
        self.year = year
        self.name = name
        self.length = length
        self.hurr_data = hurr_data

In [4]:
def load_hurr_data():
    filename = r'pickledump.txt'
    filehandler = open(filename, 'rb') 
    new_stuff = pickle.load(filehandler)
    return new_stuff

In [6]:
def create_training_data(hurr_obj, columns,  hurricane_number, window_length = 5, hurricane_year = 0):
    all_inputs = []
    all_labels = []
    for i in range(1):
        if(int(hurricanes[hurricane_number].year) > hurricane_year):
            current_df = hurricanes[hurricane_number].hurr_data
            current_begin = 0
            current_end = current_begin + window_length
            while((current_end + 1) < len(current_df)):
                hold = current_df[current_begin:current_end]
                if(not is_negative(hold[['Max_Pressure']].values) and not is_negative(hold[['Max-wind']].values)):
                    hold = hold[columns].values
                    input_vals = hold
                    labels = current_df[current_end:current_end + 1]
                    labels = labels[columns].values

                    all_inputs.append(input_vals)
                    all_labels.append(labels)
                current_begin += 1
                current_end +=1
    all_inputs = np.array(all_inputs)
    all_labels = np.array(all_labels)
    return all_inputs, all_labels

In [ ]:
def is_negative(lista):
    #print(lista)
    
    #print(lista)
    for i in range(len(lista)):
        #print(lista[i])
        if(int(lista[i]) < 0):
            return True
    return False

In [5]:
def add_single_lines(df, window_length, columns):
    label_start = window_length
    label_end = len(df)
    input_start = 0
    input_end = len(df) - 1

    return df[label_start:label_end][columns].values, df[input_start:input_end][columns].values

In [131]:
def adjust_test_results(train, predicted, actual):
    results = []
    for i in range(len(predicted)):
        new_lat, new_long = add_distance(train[i][window_length - 1][0], train[i][window_length - 1][1], predicted[i][0], predicted[i][1])
        results.append([new_lat, new_long])
        #print(new_lat, new_long)
        
    results = np.array(results)
    actual = actual[:,0,:]
    #return results, actual
    return mean_absolute_error(results, actual[:,0:2])

In [ ]:
def adjust_training_test(train_inputs, train_labels, window_length):
    new_labels = []
    train_labels = train_labels[:,0,:]
    for i in range(len(train_inputs)):
        dist, bearing = distance(train_inputs[i][window_length - 1][0], train_inputs[i][window_length - 1][1], train_labels[i][0], train_labels[i][1])
        new_line = [dist, bearing, train_labels[i][2], train_labels[i][3]]
        new_labels.append(new_line)
    return np.array(new_labels)       

In [ ]:
hurricanes = load_hurr_data()

In [7]:
train_inputs, train_labels, test_inputs, test_labels, input_train_lines, output_train_lines, input_test_lines, input_train_lines = find_training_test(hurricanes, ['Latitude', 'Longitude', 'Max-wind', 'Max_Pressure'], 5, 1)

NameError: name 'find_training_test' is not defined

In [122]:
np.vstack([[0,1,2], [3,4,5]])

array([[0, 1, 2],
       [3, 4, 5]])

In [ ]:
test_labels1[:,0:2]

In [31]:
print(mseMLR1, mseMLR2, mse3)

0.22835758410181617 41.165016459663676 0.764488066051902


In [30]:
window_length = 5
mseMLR1, model, pred2 = non_RNN_models(train_inputs, train_labels, test_inputs, test_labels, 'MLR')
new_train_labels = adjust_training_test(train_inputs, train_labels, window_length)
new_test_labels = adjust_training_test(test_inputs, test_labels, window_length)


mseMLR2, model, pred2 = non_RNN_models(train_inputs, new_train_labels, test_inputs, new_test_labels, 'MLR', adjustment_needed = False)
mse3 = adjust_test_results(test_inputs, pred2, test_labels)

In [145]:
print(train_inputs[0][window_length - 1][0], train_inputs[0][window_length - 1][1], train_labels[0], train_labels[1])

27.2 72.5 [[  28.    73.6   30.  1010. ]] [[  28.9   74.3   30.  1007. ]]


In [9]:
def huber_approx_obj(preds, dtrain):

    d = preds - dtrain.get_labels() #remove .get_labels() for sklearn

    h = 1  #h is delta in the graphic

    scale = 1 + (d / h) ** 2

    scale_sqrt = np.sqrt(scale)

    grad = d / scale_sqrt

    hess = 1 / scale / scale_sqrt

    return grad, hess

In [10]:
def non_RNN_models(train_inputs, train_labels, test_inputs, test_labels, model_type, adjustment_needed = True):
    if(model_type == 'DT'):
        regressor = DecisionTreeRegressor(random_state = 42)
    elif(model_type == 'MLR'):
        regressor = LinearRegression()
    elif(model_type == 'XGBOOST'):
        xgb = XGBRegressor(objective='reg:squarederror')
        regressor = MultiOutputRegressor(xgb)
    
    train_inputs = (train_inputs.transpose(0,2,1).reshape(-1,20)) 
    test_inputs = (test_inputs.transpose(0,2,1).reshape(-1,20)) 
    if(adjustment_needed == True):
        test_labels = test_labels[:,0,:]
        train_labels = train_labels[:,0,:]
        
    regressor.fit(train_inputs, train_labels)
    y_pred = regressor.predict(test_inputs)
    MSE = mean_absolute_error(test_labels[:,0:2], y_pred[:,0:2])
    return MSE, regressor, y_pred

In [133]:
def find_training_test(hurr_objs, columns, window_length, return_dim):
    all_hurrs = []
    for i in range(len(hurr_objs)):
        if(hurr_objs[i].length  > window_length and not is_negative(hurr_objs[i].hurr_data[['Max_Pressure']].values) and not is_negative(hurr_objs[i].hurr_data[['Max-wind']].values)):
            all_hurrs.append(i)
           
    train_hurrs, test_hurrs = train_test_split(all_hurrs,  test_size=0.20, random_state=42)
    #if(return_dim == 2):
    train_inputs = []
    train_labels = []
    test_inputs = []
    test_labels = []
    all_input_train_lines = []
    all_output_train_lines = []
    all_input_test_lines = []
    all_output_test_lines = []
    print(len(hurr_objs))
    for i in range(len(hurr_objs)):
        print(i)
        if(i in train_hurrs):
            current_df = hurr_objs[i].hurr_data
            current_begin = 0
            current_end = current_begin + window_length
            while((current_end + 1) < len(current_df)):
                hold = current_df[current_begin:current_end]
                if(not is_negative(hold[['Max_Pressure']].values) and not is_negative(hold[['Max-wind']].values)):
                    hold = hold[columns].values
                    input_vals = hold
                    labels = current_df[current_end:current_end + 1]
                    labels = labels[columns].values

                    train_inputs.append(input_vals)
                    train_labels.append(labels)
                output_lines, input_lines = add_single_lines(current_df, window_length, columns)
                all_input_train_lines.append(input_lines)
                all_output_train_lines.append(output_lines)
                current_begin += 1
                current_end +=1
        elif(i in test_hurrs):
            current_df = hurr_objs[i].hurr_data
            current_begin = 0
            current_end = current_begin + window_length
            while((current_end + 1) < len(current_df)):
                hold = current_df[current_begin:current_end]
                if(not is_negative(hold[['Max_Pressure']].values) and not is_negative(hold[['Max-wind']].values)):
                    hold = hold[columns].values
                    input_vals = hold
                    labels = current_df[current_end:current_end + 1]
                    labels = labels[columns].values

                    test_inputs.append(input_vals)
                    test_labels.append(labels)
                    
                output_lines, input_lines = add_single_lines(current_df, window_length, columns)
                all_input_test_lines.append(input_lines)
                all_output_test_lines.append(output_lines)
                current_begin += 1
                current_end +=1
        
    train_inputs = np.array(train_inputs)
    train_labels = np.array(train_labels)
    test_inputs = np.array(test_inputs)
    test_labels = np.array(test_labels)
    all_input_train_lines = np.vstack(all_input_train_lines)
    all_output_train_lines = np.vstack(all_output_train_lines)
    all_input_test_lines = np.vstack(all_input_test_lines)
    all_output_test_lines = np.vstack(all_output_test_lines)
    return train_inputs, train_labels, test_inputs, test_labels, all_input_train_lines, all_output_train_lines, all_input_test_lines, all_output_test_lines

In [223]:
norm_train_inputs, norm_test_inputs,norm_train_outputs, scalar_test, scalar_train = normalize_train(input_train_lines, input_test_lines, train_inputs, test_inputs, train_labels)

In [222]:
def normalize_train(input_train_lines, input_test_lines, train_inputs, test_inputs, train_outputs):
    values = input_train_lines
    #values = values.reshape((len(values), 1))
    # train the standardization
    scaler = StandardScaler()
    scaler = scaler.fit(values[:,0:2])
    #print('Mean: %f, StandardDeviation: %f' % (scaler.mean_, sqrt(scaler.var_)))
    # standardization the dataset and print the first 5 rows
    all_norms_train = []
    all_norms_output_train = []
    all_norms_test = []
    scaler_out = StandardScaler()
    scaler_out = scaler_out.fit(input_test_lines[:,0:2])
    for i in range(len(train_inputs)):
        normalized_train = scaler.transform(train_inputs[i,:,0:2])
        normalized_out_train = scaler.transform(train_outputs[i,:,0:2])
        all_norms_train.append(normalized_train)
        all_norms_output_train.append(normalized_out_train)
    for i in range(len(test_inputs)):
        normalized_test =scaler_out.transform(test_inputs[i,:,0:2])
        all_norms_test.append(normalized_test)
    return np.array(all_norms_train), np.array(all_norms_test), np.array(all_norms_output_train), scaler, scaler_out



In [ ]:

for i in range(5):
	print(normalized[i])
# inverse transform and print the first 5 rows
inversed = scaler.inverse_transform(normalized)
for i in range(5):
	print(inversed[i])
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
# Standardize time series data
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from math import sqrt
# load the dataset and print the first 5 rows
series = read_csv('daily-minimum-temperatures-in-me.csv', header=0, index_col=0)
print(series.head())
# prepare data for standardization
values = series.values
values = values.reshape((len(values), 1))
# train the standardization
scaler = StandardScaler()
scaler = scaler.fit(values)
print('Mean: %f, StandardDeviation: %f' % (scaler.mean_, sqrt(scaler.var_)))
# standardization the dataset and print the first 5 rows
normalized = scaler.transform(values)
for i in range(5):
	print(normalized[i])
# inverse transform and print the first 5 rows
inversed = scaler.inverse_transform(normalized)
for i in range(5):
	print(inversed[i])

In [229]:
def create_RNN(train_inputs, train_labels, test_inputs, test_labels, window_length, scaler_train, scaler_test, adjustment_needed = False):
    #train_inputs = (train_inputs.transpose(0,2,1).reshape(-1,20))[:,np.newaxis, :]
    #test_inputs = (test_inputs.transpose(0,2,1).reshape(-1,20))[:,np.newaxis, :]
    print(train_inputs.shape)
    print(test_inputs.shape)
    #return train_inputs, None, None
    if(adjustment_needed == True):
        test_labels = test_labels[:,0,:]
        train_labels = train_labels[:,0,:]
    print(test_labels.shape)
    print(train_labels.shape)
    #return train_inputs, train_labels, test_inputs, test_labels
    model = Sequential()
    model.add(LSTM(units=128,return_sequences = True, input_shape=(5,2)))
    model.add(Dropout(0.3))
    model.add(LSTM(units=128,return_sequences = False, input_shape=(5,2)))
    model.add(Dropout(0.3))
    #model.add(Dense(units=32, activation = 'relu'))
    #model.add(LSTM(units=128, activation = 'relu'))
    model.add(Dense(units=2, activation = 'relu'))
    #model.summary()
    sgd = keras.optimizers.rmsprop(learning_rate=0.01)#, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(optimizer=sgd, loss='mean_absolute_error')
    i = 100000000
    model.fit(train_inputs[:,:, 0:2], train_labels[:,0:2], epochs=15, batch_size=32)
    y_pred= model.predict(test_inputs[:,:,0:2])
    y_pred = scaler_test.inverse_transform(y_pred)
    # Save the model
    model.save('rnnv2.h5')
    mae = mean_absolute_error(test_labels[:,0:2], y_pred[:,0:2])

    return mae, test_labels, y_pred

In [230]:
#new_train_labels = adjust_training_test(train_inputs, train_labels, window_length)
#new_test_labels = adjust_training_test(test_inputs, test_labels, window_length)

mae, model, y_pred = create_RNN(norm_train_inputs, norm_train_outputs,norm_test_inputs, test_labels, window_length,scalar_train, scalar_test, adjustment_needed = True)

(10755, 5, 2)
(2837, 5, 2)
(2837, 4)
(10755, 2)
Epoch 1/15
10755/10755 [==============================] - 14s 1ms/step - loss: 0.5371
Epoch 2/15
10755/10755 [==============================] - 13s 1ms/step - loss: 0.4651
Epoch 3/15
10755/10755 [==============================] - 14s 1ms/step - loss: 0.4498
Epoch 4/15
10755/10755 [==============================] - 13s 1ms/step - loss: 0.4424
Epoch 5/15
10755/10755 [==============================] - 12s 1ms/step - loss: 0.4375
Epoch 6/15
10755/10755 [==============================] - 13s 1ms/step - loss: 0.4348: 0s - 
Epoch 7/15
10755/10755 [==============================] - 14s 1ms/step - loss: 0.4329
Epoch 8/15
10755/10755 [==============================] - 16s 1ms/step - loss: 0.4310
Epoch 9/15
10755/10755 [==============================] - 15s 1ms/step - loss: 0.4301
Epoch 10/15
10755/10755 [==============================] - 13s 1ms/step - loss: 0.4280: 0s - lo
Epoch 11/15
10755/10755 [==============================] - 15s 1ms/step - l

In [374]:
hold1, hold2, hold3, hold4 = create_RNN(train_inputs, train_labels, test_inputs, test_labels, window_length, adjustment_needed = True)

(10755, 1, 20)
(2837, 1, 20)
(2837, 4)
(10755, 4)


In [175]:
mse3 = adjust_test_results((test_inputs.transpose(0,2,1).reshape(-1,20)),  y_pred, test_labels)

IndexError: invalid index to scalar variable.

In [540]:
K.clear_session()

In [14]:
hurricanes = load_hurr_data()
print(len(hurricanes))

1864


In [232]:
y_pred[0:5]

array([[32.863678, 77.99661 ],
       [33.603443, 77.670494],
       [34.499813, 76.58016 ],
       [35.084187, 75.102135],
       [35.261593, 73.594696]], dtype=float32)

In [233]:
test_labels[0:5]

array([[[  32.4,   78.7,   25. , 1012. ]],

       [[  33.3,   78. ,   25. , 1011. ]],

       [[  34. ,   77. ,   30. , 1006. ]],

       [[  34.4,   75.8,   35. , 1004. ]],

       [[  34. ,   74.8,   40. , 1002. ]]])

In [382]:
mean_absolute_error(model[:,0:2], y_pred[:,0:2])

15.840175564422722

In [520]:
mse3 = adjust_test_results(test_inputs, model, y_pred)

In [231]:
mae

6.506329965104001

In [189]:
norm_train_inputs[:, :, 0:2]

array([[[-0.23453956,  0.40932324],
        [-0.23453956,  0.46389391],
        [-0.22527359,  0.51350361],
        [-0.18820969,  0.56311332],
        [-0.12334787,  0.61272302]],

       [[-0.23453956,  0.46389391],
        [-0.22527359,  0.51350361],
        [-0.18820969,  0.56311332],
        [-0.12334787,  0.61272302],
        [-0.04922008,  0.6672937 ]],

       [[-0.22527359,  0.51350361],
        [-0.18820969,  0.56311332],
        [-0.12334787,  0.61272302],
        [-0.04922008,  0.6672937 ],
        [ 0.03417369,  0.70202049]],

       ...,

       [[ 2.30433738, -1.34189931],
        [ 2.37846517, -1.47584551],
        [ 2.48039089, -1.65444044],
        [ 2.60084855, -1.83799634],
        [ 2.72130622, -2.01163031]],

       [[ 2.37846517, -1.47584551],
        [ 2.48039089, -1.65444044],
        [ 2.60084855, -1.83799634],
        [ 2.72130622, -2.01163031],
        [ 2.81396596, -2.13565456]],

       [[ 2.48039089, -1.65444044],
        [ 2.60084855, -1.83799634],
     

In [358]:
model[index]

array([  34.,   77.,   30., 1006.])

In [522]:
mse3

14.348331115522512

In [354]:
hold1

array([[[  26. ,   26. ,   26.1, ..., 1014. , 1013. , 1012. ]],

       [[  26. ,   26.1,   26.5, ..., 1013. , 1012. , 1010. ]],

       [[  26.1,   26.5,   27.2, ..., 1012. , 1010. , 1007. ]],

       ...,

       [[  53.4,   54.2,   55.3, ...,  952. ,  956. ,  960. ]],

       [[  54.2,   55.3,   56.6, ...,  956. ,  960. ,  964. ]],

       [[  55.3,   56.6,   57.9, ...,  960. ,  964. ,  968. ]]])

In [26]:
def distance(s_lat, s_lng, e_lat, e_lng):

   # approximate radius of earth in km
    R = 6373.0

    s_lat = s_lat*np.pi/180.0                      
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  

    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2
    
    bearing = get_bearing(s_lat, s_lng, e_lat, e_lng)

    return 2 * R * np.arcsin(np.sqrt(d)), bearing

In [27]:
def get_bearing(lat1, long1, lat2, long2):
    dLon = (long2 - long1)

    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)

    brng = math.atan2(y, x)

    brng = np.rad2deg(brng)

    return brng

In [28]:
def add_distance( lat, lon, bearing, distance):
    EARTH_RADIUS = 6371.00
    # convert Latitude and Longitude
    # into radians for calculation
    latitude = math.radians(lat)
    longitute = math.radians(lon)

    # calculate next latitude
    next_latitude = math.asin(math.sin(latitude) *
                    math.cos(distance/EARTH_RADIUS) +
                    math.cos(latitude) *
                    math.sin(distance/EARTH_RADIUS) *
                    math.cos(math.radians(bearing)))

    # calculate next longitude
    next_longitude = longitute + (math.atan2(math.sin(math.radians(bearing)) *
                                             math.sin(distance/EARTH_RADIUS) *
                                             math.cos(latitude),
                                             math.cos(distance/EARTH_RADIUS) -
                                             math.sin(latitude) *
                                             math.sin(next_latitude)
                                            )
                                 )

    # convert points into decimal degrees
    new_lat = math.degrees(next_latitude)
    new_lon = math.degrees(next_longitude)

    return new_lat, new_lon

In [29]:
def create_map_input_MLR(df_hurr, hurr_num):
    inputs, labels = create_training_data(hurricanes, ['Latitude', 'Longitude', 'Max-wind', 'Max_Pressure'], 1700)
    actual_pred, y_pred, MSE = MLR_create_and_pred(inputs)
    output = hold
    df_hurr = df_hurr[hurr_num].hurr_data
    df_hurr = df_hurr[['Latitude', 'Longitude']]
    df_output = pd.DataFrame({
        'Latitude':output[:, 0],
        'Longitude':output[:, 1]
    })
    Visualization.plot_hurricanes(df_output, df_hurr)
def create_map_input_XGBOOST(df_hurr, hurr_num):
    inputs, labels = create_training_data(hurricanes, ['Latitude', 'Longitude', 'Max-wind', 'Max_Pressure'], 1700)
    hold = XGBOOST.load_and_predict(1,inputs)
    output = hold
    df_hurr = df_hurr[hurr_num].hurr_data
    df_hurr = df_hurr[['Latitude', 'Longitude']]
    df_output = pd.DataFrame({
        'Latitude':output[:, 0],
        'Longitude':output[:, 1]
    })
    Visualization.plot_hurricanes(df_output, df_hurr)
def create_map_input_Decision(df_hurr, hurr_num):
    inputs, labels = create_training_data(hurricanes, ['Latitude', 'Longitude', 'Max-wind', 'Max_Pressure'], 1700)
    hold, hold1, hold2 = Decision_Tree.create_and_pred(inputs)
    output = hold
    df_hurr = df_hurr[hurr_num].hurr_data
    df_hurr = df_hurr[['Latitude', 'Longitude']]
    df_output = pd.DataFrame({
        'Latitude':output[:, 0],
        'Longitude':output[:, 1]
    })
    Visualization.plot_hurricanes(df_output, df_hurr)
    return hold    

In [227]:
#create_map_input_MLoR(hurricanes, 1700)
#create_map_input_MLR(hurricanes, 1700)
#create_map_input_XGBOOST(hurricanes, 1700)
#hold = create_map_input_Decision(hurricanes, 1700)

In [9]:
filename = '1970-present_OFCL_v_BCD5_ind_ATL_TI_errors_noTDs.txt.txt'

In [10]:

df4 = pd.read_csv(filename, delimiter='\s+')

In [11]:
(df4[df4['STMID'] == 'AL112008'][['Date/Time','Lat', 'Lon','000hT01','012hT01', '024hT01']])

,Date/Time,Lat,Lon,000hT01,012hT01,024hT01
8163,25-09-2008/06:00:00,22.0,69.4,-9999.0,-9999.0,-9999.0
8164,25-09-2008/12:00:00,22.4,68.7,-9999.0,-9999.0,-9999.0
8165,25-09-2008/18:00:00,23.1,68.4,0.0,13.2,48.3
8166,26-09-2008/00:00:00,24.0,68.0,12.0,40.3,30.5
8167,26-09-2008/06:00:00,25.0,68.2,0.0,37.5,37.5
8168,26-09-2008/12:00:00,26.0,68.6,5.4,42.5,36.3
8169,26-09-2008/18:00:00,26.9,68.6,0.0,24.0,43.2
8170,27-09-2008/00:00:00,28.7,68.6,0.0,31.3,45.6
8171,27-09-2008/06:00:00,29.9,69.3,7.9,18.0,36.3
8172,27-09-2008/12:00:00,31.2,69.5,0.0,36.3,48.6
